In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
import cv2
import keras 
from keras.utils import normalize
from keras import *
import pandas as pd
import tensorflow as tf


In [ ]:
# import the pre-trained model
from tensorflow.python.keras.models import load_model
pre_trained_model = load_model('U_Net_SA1_2.h5')


In [ ]:
# Check that all the layers in the pre-trained model are in default trainable
layers = [(layer, layer.name, layer.trainable) for layer in pre_trained_model.layers]
pre_trained_model_df = pd.DataFrame(layers, columns=["Layer Type", "Layer Name", "Layer Trainable"])

display(pre_trained_model_df)


In [ ]:
pre_trained_model.summary()


In [ ]:
# Freeze the all the convolutional layers except the last decoding block
for layer in pre_trained_model.layers:
    if layer.name in ["conv2d_38", "conv2d_39","conv2d_40","conv2d_41","conv2d_42","conv2d_43","conv2d_44",
                     "conv2d_45","conv2d_46","conv2d_47",
                     "conv2d_transpose_8","conv2d_48","conv2d_49","conv2d_transpose_9",
                     "conv2d_50","conv2d_51","conv2d_transpose_10",
                     "conv2d_52","conv2d_53"]:
        layer.trainable = False


In [ ]:
# display the trainable and untrainable layers
layers = [(layer, layer.name, layer.trainable) for layer in pre_trained_model.layers]
pre_trained_model_df = pd.DataFrame(layers, columns=["Layer Type", "Layer Name", "Layer Trainable"])
pd.set_option('display.max_rows', pre_trained_model_df.shape[0]+1)
pre_trained_model_df


In [ ]:
# Training and validation data from SA0

In [ ]:
X_train_val_SA0=np.load('X_array_SA0.npy')

In [ ]:
y_train_val_SA0=np.load('y_array_SA0.npy')

In [ ]:
# split the data into training and validation
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_train_val_SA0,y_train_val_SA0=0.2,random_state=42)

In [ ]:
# Build the model
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 13

In [ ]:
optimizer = tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')

pre_trained_model.compile(optimizer=optimizer, loss=['mse'], loss_weights=[1],metrics=['accuracy'])
pre_trained_model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,TensorBoard
from datetime import datetime
start_time = datetime.now()
NAME="U_Net_SA1_2_SA0"
checkpointer = ModelCheckpoint("U_Net_SA1_2_SA0.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))


history = pre_trained_model.fit(x_train, y_train, validation_split=0.25, batch_size=10, epochs=500, callbacks=[checkpointer,early,tensorboard])
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
